In [1]:
import pandas as pd
from requests_html import HTMLSession
import os #lets pandas talk to the OS 
import re
from time import sleep
from tqdm.notebook import tqdm # import progress bar


#### SECTION 1: get the urls to crawl (or data that can build the urls)

if not os.path.exists('input/sp500_urls.csv'):

    # open page
    session = HTMLSession() 
    url = 'https://en.m.wikipedia.org/wiki/List_of_S%26P_500_companies'
    r = session.get(url)

    # find the links
    table = r.html.find('#constituents')[0]  # reduce to the table
    table_rows = table.find('tr')[1:]
    colinks = []
    for row in table_rows:
        a_url =   list(row.find('td')[1].absolute_links)[0]    
        colinks.append(a_url)

    # save list to csv    
    if not os.path.exists('input/'):
        os.makedirs('input/')
    pd.DataFrame(colinks).to_csv('input/sp500_urls.csv',index=False,header=False)
    
else:
    colinks = pd.read_csv('input/sp500_urls.csv',names=['firm']) ['firm'].to_list()


#### SECTION 2: crawl the urls

# for url in colinks[:10]: first 10 firms 
for url in tqdm(colinks,total=len(colinks)):
    
    folder = 'wiki_html/'
    if not os.path.exists('wiki_html/'):
        os.makedirs('wiki_html/')
    
    name = re.sub( r'\W+','',     url.split('/')[-1]    )
    filepath = folder + name + '.txt'
        
    if not os.path.exists(filepath):
        
        try:
            r = session.get(url)
        except:
            print('sad face failure:',url)
        else:   
            with open(filepath,'w',encoding='utf8') as f:
                f.write(r.text)
            sleep(2) # be nice to server        


In [2]:
type(colinks)

list